In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import requests 
import bs4 
import nltk.data
from datetime import datetime
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [2]:
data = pd.read_csv('daily_signal_full_text.csv')
data['text'] = data['text'].apply(lambda x: x.strip() if isinstance(x, str) else 0)

In [15]:
data = data[data['text']!=0] ## excluding graphics
data = data.reset_index(drop=True)

In [4]:
data['Sent'] = data['text'].apply(lambda x: [j for i in x.split('\n') 
                                             for j in tokenizer.tokenize(i)])

In [25]:
sent = data['Sent'].sum()
count = Counter(sent)

In [51]:
drop_words = ['Photo', 'java', '…', '***', 'http', 'if(', 'for(',
              'Photo credit:', 'Morning Bell', 'Read more', 'Originally appeared',
              'For more information', 'podcast', 'leave a review',
              'leave us a message', 'COPYRIGHT', 'DISTRIBUTED', 'Originally published',
              'email protected', 'LEARN MORE', 'Quick Hits', 'Newscom', 'Click here',
              'We also cover these stories', 'Opportunity Scholarship Program',
              'This article has been modified', '[CDATA[', 'General_URL', 'Read More']
drop_sent = []
for i in count:
    if len(i.split('\\')) > 2: drop_sent.append(i)
    if len(i.split('/')) > 2: drop_sent.append(i)
    if len(i.split('='))  > 2: drop_sent.append(i)
    if len(i.split('\t')) > 2: drop_sent.append(i)
    if len(i.split('}'))  > 2: drop_sent.append(i)
    for j in drop_words:
        if j in i: 
            drop_sent.append(i)
            continue

In [52]:
data['cleaned_sent'] = data['Sent'].apply(lambda x: 
                                          [i for i in x if i not in drop_sent])

In [56]:
data['text'] = data['cleaned_sent'].apply(lambda x: "\n".join(x))

In [58]:
data = data[['urls', 'category', 'date', 'text', 'authors']]

In [60]:
data.to_csv('signal_cleaned_text.csv', index=False)